In [4]:
import z3
import sys
import tree_sitter
from tree_sitter import Language, Parser, Tree, Node
import tree_sitter_c
import os
from z3 import Int, Real, Bool, Solver, And, sat, simplify, is_int_value, is_expr, substitute
import numpy as np

In [5]:

class SymbolicState:
    
    def __init__(self):
         # Store variable names as keys and types as values
        self.variables = {} 
        #Store the values 
        self.values = {}
         # Store path conditions for the state
        self.path_conditions = [] 
 

    def add_variable(self, name, var_type, value=None):
        self.variables[name] = var_type  # Store variable type
        if value is None:
            value = z3.Int(name + "_0")
        
        self.values[name] = value

        
    def get_variable(self, name):
        # Retrieve the current value of the variable
        return self.values.get(name, None)  

    def update_variable(self, name, value):
        if name in self.values:
            self.values[name] = value  # Update the current value
        else:
            raise Exception(f"Variable '{name}' not found in the current state.")
    
    def add_constraint(self, condition):
        """Add a path condition to the state."""
        self.path_conditions.append(condition)
        
        
    def get_constraints(self):
        return self.path_conditions
        
    def get_all_variables(self):
        return self.variables
    
    def __repr__(self):
        return f"SymbolicState(variables={self.variables}, path_conditions={self.path_conditions})"

In [22]:

class SymbolicExecutionEngine:
    
    def __init__(self, source_code, function_name):
        
        #parsing the source code
        self.C_LANG = None 
        C_LANG = Language(tree_sitter_c.language())
        self.parser = Parser(C_LANG)                  
        #self.tree = self.parse_c_file(source_code)
        self.ast = self.parser.parse(source_code.encode())
        
        #attributes  
        self.loop_states = []
        self.target_reached = False
        self.target_reached_conditions = []
        self.function_name = function_name
        self.state_stack = []
        self.target_reachable_state = None
        self.infeasible_paths = 0
        self.feasible_paths = 0
        self.conditions_safe = []
        
    
    
    def negate_last_condition(self):
        
        condition = self.conditions_safe[-1]
        self.conditions_safe[-1] = z3.Not(condition)
        
    def is_feasible(self, condition):
        

        print("\n Inside Is feasible Function")
        current_state = self.get_current_state()
        
        solver = Solver()
        
        #add all previous states
        for c in self.conditions_safe:
            print(c)
            solver.add(c)
        

        # Convert current_state values to substitution pairs
        substitution_pairs = []
        for from_expr, to_expr in current_state.values.items():
            # Ensure both from_expr and to_expr are Z3 expressions
            substitution_pairs.append((z3.Int(from_expr), to_expr))

        # Debug: Print substitution pairs to confirm structure
        #print("Substitution Pairs:", substitution_pairs)

        # Apply substitution to each constraint
        #for con in current_state.get_constraints():
        #    if is_expr(con):
        #        try:
        #            con_substituted = substitute(con, *substitution_pairs)
        #            print("Substituted Condition:", con_substituted)
        #            solver.add(con_substituted)
        #        except Exception as e:
        #            print("Error during substitution:", e)
        #    else:
        #        print(f"Condition is not Z3 expression: {con}")
        #        print(f"Condition: {con}, type: {type(con)}")

        # Applying substitutions to additional conditions in `condition`
        con = condition[-1]
        if is_expr(con):
            try:
                con_substituted = substitute(con, *substitution_pairs)
                print("Substituted Condition:", con_substituted)
                solver.add(con_substituted)
            except Exception as e:
                print("Error during substitution in `condition` list:", e)
        else:
            print(f"Condition is not Z3 expression: {con}")
            print(f"Condition: {con}, type: {type(con)}")
                              
        # Check satisfiability
        if solver.check() == sat:
            self.conditions_safe.append(con_substituted)
            return True


    def add_state(self, state):
        self.state_stack.append(state)
    
    
    def get_concrete_values(self):
        concrete_values = []
        solver = Solver()

        # Assume self.target_reachable_state.variables is a dictionary mapping variable names to Z3 variables
        variable_names_nonZ3 = list(self.target_reachable_state.variables.keys())  
        #print(variable_names_nonZ3)
        
        variable_names_Z3 = []
        for var in variable_names_nonZ3:
            variable_names_Z3.append(z3.Int(var + "_0"))
        
        #print(variable_names_Z3)

        # Add target reached conditions as constraints to the solver
        for con in self.target_reached_conditions:
            solver.add(con)

        # Check satisfiability
        if solver.check() == sat:
            model = solver.model()  # Retrieve the model with concrete values
            for var in variable_names_Z3:
                concValue = model.evaluate(var)  # Directly evaluate var in the model
                print(f"variable {var} | Concrete Value: {concValue}")
                concrete_values.append(concValue)
            return concrete_values
        else:
            print("NOT SAT")
            print("Unsatisfiable constraints:", solver.unsat_core())

    
    def output_results(self):
        print(f"Total Number of infeasible states: {self.infeasible_paths}")
        print(f"Total Number of feasible states through function: {self.feasible_paths}")

        if(self.target_reached):
            
            # Target statement analysis
            print(f"Number of feasible paths to reach target statement:")
            print(len(self.target_reachable_state.path_conditions))
            
            
            print("Constraints to reach target statement:")
            for constraint in self.target_reachable_state.path_conditions:
                print(constraint)
        
        
            print("Concrete values to reach target statement:")
            self.get_concrete_values()
        
    
    def find_variable(self, variable_name):
        current_state = self.get_current_state() if self.state_stack else None

        #print('\n Inside finding variable from state')
        
        # Check if there is a current state
        if current_state is None:
            raise ValueError("No current state available.")

        # Check if the variable exists in the current state's variables
        if variable_name in current_state.variables:
            variable_type = current_state.variables[variable_name]
            #print(f"variable_type: {variable_type}")
            return variable_name, variable_type  # Return the variable name and its type

        # If the variable is not found, return None
        return None    
    
    def push_state(self, new_state):
        # Push a new symbolic state onto the stack
        self.state_stack.append(new_state)

    def pop_state(self):
        # Pop the last symbolic state from the stack
        if self.state_stack:
            return self.state_stack.pop()
        else:
            raise Exception("State stack is empty.")

            
    def is_empty(self):
        # Check if the state stack is empty
        return len(self.state_stack) == 0
    
    
    def get_current_state(self):
        # Get the current symbolic state without popping it
        if self.state_stack:
            return self.state_stack[-1]
        else:
            raise Exception("State stack is empty.")

    def print_states(self):
        # For debugging: print all current states in the stack
        for i, state in enumerate(self.state_stack):
            print(f"State {i}:", state.variables, state.values, state.path_conditions)       
    
    def print_current_state(self):
        state = self.get_current_state()
        print(f"State:", state.variables, state.values, state.path_conditions)   
    
    def print_ast(self,node, indent=0):
        print("  " * indent + f"{node.type}")
        for child in node.children:
            self.print_ast(child, indent + 1)
        
    
    def print_node(self,node: Node, indent=0):
        if node.children:
            print("<details open>")
            print(f"<summary>{node.type} | {node.text}</summary>")

            # print(' ' * indent + f'Node(type={node.type!r}, start_byte={node.start_byte}, end_byte={node.end_byte})')
            #for child in node.children:
            #    print_node(child, indent + 2)

            #print("</details>")
        else:
            print(f"<div>{node.type}</div>")


    def parse_c_file(self, content: str) -> Tree:
    
        if self.C_LANG is None:
            C_LANG = Language(tree_sitter_c.language())

        parser = Parser(C_LANG)
        return parser.parse(content.encode('utf-8'))
        
    
    def interpret(self):
        # Entry point to start symbolic execution
        self._symbolically_execute_function(self.function_name)
    
    def _symbolically_execute_function(self, function_name):
        # Find the function in the AST and start executing
        # Traverse the AST to find the function definition
        root_node = self.ast.root_node
        
        function_node = self._find_function_node(root_node, function_name)
        if function_node:
            print('Founnd the Function')
            print(f"Function node type: {function_node.type}")
            
            #first go to compount sttement, and then pass that to execute fucntion
            
            # Attempt to find compound statements within the node
            compound_stmt = self.find_Compound_statements_inNode(function_node)

            if compound_stmt:
                print(f"Found compound statement of type: {compound_stmt.type}")
                self._execute_function_statements(compound_stmt)
            else:
                print("No compound statement found to execute.")
    
    
    #this is helper function to find the function
    def _find_function_node(self, root, name):
        # Traverse the AST to find the function by name
        for node in root.children:
            if node.type == "function_definition" and self._function_name(node) == name:
                return node
        return None
    
    
    #this is helper function to find the function
    def _function_name(self, node):
        # Traverse the function_declarator to find the identifier (function name)
        for child in node.children:
            if child.type == "function_declarator":
                for sub_child in child.children:
                    if sub_child.type == "identifier":
                        return sub_child.text.decode('utf-8')  # Extract and return function name
        raise ValueError("Function name node not found in the AST.")
    
    
    
    # this is helper fucntion, because all logic of fucntion
    # is inside the compound statement
    def find_Compound_statements_inNode(self, node):
        
        # Iterate through the children of the current node
        for child in node.children:
            print(f"child node type: {child.type}")

            if child.type == "function_declarator":
                print("Found function definition. To find if it has parameters")
                self.handle_function_params(child)
                
            # First, check if the current node is a "compound_statement"
            if child.type == "compound_statement":
                print("Found compound statement. Now processing")
                return child
            
    
    # if function has parameters 
    def handle_function_params(self, node):
        # Print the function's parameter list text
        print(f"Parameter: {node.text}")

        parameters_node = node.child_by_field_name("parameters")
        if parameters_node:
            print("Parameters found:")

            # Iterate over each child parameter in the parameters node
            for param in parameters_node.children:
                if param.type == "parameter_declaration":
                    # Retrieve the type and name fields of the parameter
                    param_type = param.child_by_field_name("type")
                    param_name = param.child_by_field_name("declarator")

                    # Decode the type and name as text if they exist
                    param_type_text = param_type.text.decode('utf-8') if param_type else "Unknown"
                    param_name_text = param_name.text.decode('utf-8') if param_name else "Unnamed"

                    # Print the parameter details
                    print(f"Parameter: {param_name_text} of type {param_type_text}")

                    # Update the symbolic state with the variable name and type
                    current_state = SymbolicState()
                    current_state.add_variable(param_name_text, param_type_text)
                    self.push_state(current_state)
                    

        else:
            print("No parameters found.")
        self.print_states()
            
    
    #When we found the function, now we will execute statements inside function
    def _execute_function_statements(self, node):
        
        for statement in node.children:
            print(f"Processing statement of type: {statement.type}")

            if statement.type == "expression_statement":
                self.handle_expression(statement)
            elif statement.type == "return_statement":
                self.handle_return(statement)
                    
            elif statement.type == "if_statement":
                self.handle_if(statement)
            elif statement.type == "while_statement":
                self.handle_while(statement)
            elif statement.type == "declaration":
                self.handle_variable_declaration(statement)
            else:
                print(f"Unsupported statement type: {statement.type}")

  
    def handle_return(self, node):
        
        if len(node.children) < 2:
            raise Exception("Return statement does not have enough children.")

        # The second child should be the number literal (return value)
        return_expr = node.children[1]  # access the number literal node

        # Interpret the return expression (it should be a number literal)
        return_value = self._interpret_expression(return_expr)
        

        # Check the return value
        if return_value is not None:
            # Log the feasible path and the return value
            print(f"path reached with return value: {return_value}")

            # If the return value is a concrete value, store it for analysis
            if return_value == 1:
                # Handle the path for return 1
                print("Path leading to return 1 found! This is Target State")
                self.target_reached = True
                self.target_reached_conditions = self.conditions_safe.copy()
                current_state = self.get_current_state()
                self.target_reachable_state = SymbolicState()
                self.target_reachable_state.variables = current_state.variables.copy()
                self.target_reachable_state.values = current_state.values.copy()
                self.target_reachable_state.path_conditions = current_state.path_conditions.copy()
                
                print("\nFinal State")
                self.print_current_state()
                #self.pop_state()
                # Add any logic to capture this path, e.g., appending to a list
                # self.feasible_paths.append(current_state)
            elif return_value == 0:
                # Handle the path for return 0
                print("Path leading to return 0 found! This is infeasible Path")
                #self.infeasible_paths += 1
                self.negate_last_condition()
                #self.pop_state()
                # Add logic as necessary for return 0 handling

        else:
            # If the return expression is None or unhandled, log a message
            print("Return expression was not interpretable.")            

        
    def handle_if(self, node):
        print("Inside Handle If Function")
        condition_node = node.child_by_field_name("condition")
        #print(f"Condition Node: {condition_node.text}")

        # Extract and add the condition to the current state's path
        condition = self._interpret_expression(condition_node)
        print(f"Condition: {condition}")

        current_state = self.get_current_state()  # Retrieve the current symbolic state
        #print("\n")
        self.print_current_state()
        #print("\n")

        # Ensure you're working with SymbolicState objects
        if not isinstance(current_state, SymbolicState):
            raise Exception(f"Expected SymbolicState, but got {type(current_state)}")
            


        # Create two new states for true and false branches
        true_state = SymbolicState()
        false_state = SymbolicState()

        # Carry over variables from the current state to the new states
        true_state.variables = current_state.variables.copy()
        true_state.values = current_state.values.copy()
        true_state.path_conditions = current_state.path_conditions.copy()
        
        false_state.variables = current_state.variables.copy()
        false_state.values = current_state.values.copy()
        false_state.path_conditions = current_state.path_conditions.copy()

        true_state.add_constraint(condition)  # Condition is true
        false_state.add_constraint(z3.Not(condition))  # Condition is false

        
        children = list(node.children)
        then_statement = None
        else_statement = None
        condition_index = children.index(condition_node)

        # Next child after the condition should be the 'then' statement
    
        if condition_index + 1 < len(children):
            then_statement = children[condition_index + 1]
            #print(f"Then statement: {then_statement.text}")

        # If there’s another child after the 'then' statement, consider it the 'else' statement
        if condition_index + 2 < len(children):
            else_statement = children[condition_index + 2]
            #print(f"Else statement statement: {else_statement.text}")
        
        
        # Check feasibility for the 'else' branch by negating the condition
        if self.is_feasible(true_state.path_conditions):
            print("True branch  is feasible")
            self.feasible_paths += 1   
            
            if then_statement:
                print("Processing 'then' statement")
                self.push_state(true_state)
                self._execute_function_statements(then_statement)
                self.pop_state()
            else:
                print("No 'then_statement' found.")
        else:
            print("True branch  is not feasible")
            self.infeasible_paths += 1 
            
            
        if self.is_feasible(false_state.path_conditions):
            print("False branch (else) is feasible")
            self.feasible_paths += 1   
               
            if else_statement:
                print("Processing 'else' statement")
                self.push_state(false_state)
                self.handle_else(else_statement)
                self.pop_state()
            else:
                print("No 'else_statement' found.")
        else:
            print("False branch (else) is not feasible")
            self.infeasible_paths += 1 
    
    
    
    
    def handle_else(self, node):
        
        #print("\n")
        #print(f"Else Node:{node.text}")
        statement = self.find_Compound_statements_inNode(node)
        self._execute_function_statements(statement)
        return
    

    def handle_while(self, node, max_num_of_iterations=25, current_depth=0):
        print("Inside Handle While Function")

        # Check if max recursion depth has been reached
        if current_depth >= max_num_of_iterations:
            print("Max recursion depth reached, exiting while loop handling.")
            return

        # Extract the condition node
        condition_node = node.child_by_field_name("condition")
        print(f"Condition Node: {condition_node.text}")

        # Interpret the condition for symbolic execution
        condition = self._interpret_expression(condition_node)
        print(f"Loop Condition: {condition}")

        # Retrieve the current symbolic state
        current_state = self.get_current_state()
        if not isinstance(current_state, SymbolicState):
            raise Exception(f"Expected SymbolicState, but got {type(current_state)}")

        # Create the state for the loop continuation (True path) and exit (False path)
        loop_state = SymbolicState()
        exit_state = SymbolicState()

        # Copy variables and path conditions from the current state
        loop_state.variables = current_state.variables.copy()
        loop_state.values = current_state.values.copy()
        loop_state.path_conditions = current_state.path_conditions.copy()
        exit_state.variables = current_state.variables.copy()        
        exit_state.values = current_state.values.copy()
        exit_state.path_conditions = current_state.path_conditions.copy()    

        # Set up path conditions for entering and exiting the loop
        loop_state.add_constraint(condition)
        exit_state.add_constraint(z3.Not(condition))

        # Check if the loop condition is satisfiable
        
        if not self.is_feasible(loop_state.path_conditions):
            print("Loop condition unsatisfiable, breaking out of loop.")
            self.push_state(exit_state)
            return  # Exit the recursion if condition is unsatisfiable

        # Push the loop state to execute the loop body under the condition
        self.loop_states.append(loop_state)
        
        self.push_state(loop_state)
        self._execute_function_statements(node.child_by_field_name("body"))
                

        # After executing the body, re-evaluate by recursively calling handle_while
        self.handle_while(node, max_num_of_iterations=max_num_of_iterations, current_depth=current_depth + 1)
        
        # Update exit state based on the last loop state
        if self.loop_states:
            last_loop_state = self.loop_states[-1] 
            exit_state.values =  last_loop_state.values.copy()

        # Pop loop state after handling
        self.pop_state()

        # Continue with the exit path after loop finishes
        self.push_state(exit_state)

        print("Finished handling while loop")
        
        

    def handle_variable_declaration(self, node):
        
        
        if(self.is_empty()):
            print("State Stack is empty")
            current_state = SymbolicState()
        else:
            print("State Stack is not empty")
            current_state = self.get_current_state()
            
        assignment_node = node.children[1]  
        print(f"Assignment node: {assignment_node.type} | {assignment_node.text}")

        # Now we need to break down this assignment node into the left and right sides
        # `assignment_node.children[0]` should be the variable name `y`
        variable_name = assignment_node.children[0].text.decode('utf-8')
        type_node = node.child_by_field_name("type")  # This line depends on your parser's AST
        variable_type = type_node.text.decode('utf-8') if type_node else "Unknown"  # Default to "Unknown" if type is missing
        print(f"Variable Type: {variable_type}")
        
        print(f"Variable Name: {variable_name}")

        # `assignment_node.children[2]` should be the right-hand expression `f()`
        if len(assignment_node.children) > 2:
            expression_node = assignment_node.children[2]
            print(f"Expression node: {expression_node.type} | {expression_node.text}")

            # Evaluate the right-hand expression (f() in this case)
            variable_value = self._interpret_expression(expression_node)
            if variable_value == np.inf: 
                current_state.add_variable(variable_name, variable_type)
                
            else:
                #if it was not a fucntion call then  
                #add value of variable 
                #print("\nAdding Value in Synbolic state\n")
                variable_value = z3.IntVal(variable_value)
                current_state.add_variable(variable_name, variable_type)
                current_state.update_variable(variable_name,variable_value)
                
                print(f"Variable {variable_name} updated with value: {variable_value}")
                print(f"Testing: {current_state.get_variable(variable_name)}")
                
        else:
            print(f"Variable {variable_name} declared without initialization")
            current_state.add_variable(variable_name, variable_type)  # Just add it if no initialization
            
        
        self.print_states()
        

        
    
    # we will use this later 
    def _to_z3_expr(self, variable_name):
        # First try to find the variable name in current state variables
        variable_info = self.find_variable(variable_name)

        if variable_info is not None:
            var_name, var_type = variable_info

            # Create Z3 expressions based on the variable name and type
            if var_type == 'int':
                z3_expr = z3.Int(var_name)  # Use the variable name for Z3 integer variable
                return z3_expr
            elif var_type == 'float':
                z3_expr = z3.Real(var_name)  # Use the variable name for Z3 real variable
                return z3_expr
            else:
                raise ValueError(f"Unsupported variable type: {var_type}")
        else:
             return z3.IntVal(variable_name)

            # Raise an error if the type is unsupported
        raise TypeError(f"Unsupported input type: {type(variable_name)}. Expected variable name as str, or a constant value.")

    
    
    def handle_expression(self, expression_node):

        # Here you would extract and process the expression
        # This is where you would interpret the expression
        result = self._interpret_expression(expression_node)

        # Update the current state if needed
        current_state = self.get_current_state()  # Example to get the current state
        # You might want to store the result or update the state based on the expression
        if result is not None:
            # Do something with the result, e.g., add to current state
             # Example:
            current_state.add_variable('result', result)  # This depends on your state structure

    
    
    
    
    
    def _interpret_expression(self, node):
        print(f"Node Type inside Interpret: {node.type}")
        print(f"Node Details :{node.text}")


        if node.type == "call_expression":
            print("Since its fucntion call, skipping the function call for now")
            return np.inf
        
        
        if node.type == "number_literal":
            # Convert number literal to an integer or float, depending on your requirements
            try:
                value = int(node.text)  # Try parsing as an integer
            except ValueError:
                value = float(node.text)  # If it fails, parse as a float
                print(f" Value to add to symbolic Variable: {value}")
            return value
        
        
        # Handle parenthesized expressions (e.g., (x > y))
        if node.type == 'parenthesized_expression':
            # The child inside the parentheses should be evaluated
            print(f"Inside Parathensis Expression: {node.children[1]}")
            return self._interpret_expression(node.children[1])  # Skip the parentheses

        # Handle literal values
        if node.type == 'number':
            return z3.IntVal(int(node.text.decode('utf-8')))  # Return integer value

        # Handle variable references
        elif node.type == 'identifier':

            variable_name = node.text.decode('utf-8')
            print(f"Variable Name :{variable_name}")
            
            current_state = self.get_current_state()
            if variable_name in current_state.variables:
                return variable_name

        # Handle binary operations (e.g., x == y, x + y)
        elif node.type == 'binary_expression':
            
            left_expr = self._interpret_expression(node.children[0])  # Left-hand side
            operator = node.children[1].text.decode('utf-8')  # Operator (e.g., +, ==)
            right_expr = self._interpret_expression(node.children[2])  # Right-hand side
        
            left_expr = self._to_z3_expr(left_expr)
            right_expr = self._to_z3_expr(right_expr)
            
            
            print(f"Left expression: {left_expr}, type: {type(left_expr)}")
            print(f"Right expression: {right_expr}, type: {type(right_expr)}")

            # Handle different binary operators
            if operator == '==':
                return left_expr == right_expr
            elif operator == '<':
                return left_expr < right_expr
            elif operator == '<=':
                return left_expr <= right_expr
            elif operator == '>':
                return left_expr > right_expr
            elif operator == '>=':
                return left_expr >= right_expr
            elif operator == '+':
                return left_expr + right_expr
            elif operator == '-':
                return left_expr - right_expr
            elif operator == '*':
                return left_expr * right_expr
            elif operator == '/':
                return left_expr / right_expr
            else:
                raise Exception(f"Unknown operator: {operator}")

        # Handle unary operations (e.g., -x, ++x, --x)
        elif node.type == 'expression_statement':

            child_node = node.children[0]
            #print(child_node.type)
            if child_node.type == 'update_expression' or child_node.type == 'unary_expression':
                #print(child_node.children)
                operator = child_node.children[1].text.decode('utf-8')
                print(f"operator: {operator}")
                
                if operator != '++' and operator != '--' and operator != '-':
                    print(child_node.children)
                    operator = child_node.children[0].text.decode('utf-8')
                    expression = self._interpret_expression(child_node.children[1])
                else:
                    expression = self._interpret_expression(child_node.children[0])
                
                print(f"expression: {expression}")
                
                
                
                current_state = self.get_current_state()
                value = current_state.get_variable(expression)
                
                if is_expr(value):
                    print(f"Value of expression is z3 expression: {expression}")
                    expr = simplify(value)
                    # Separate symbolic and constant parts
                    symbolic_part = None
                    constant_part = 0

                    # Check if expr is an addition or subtraction
                    if expr.decl().name() == 'sub' or expr.decl().name() == 'add':
                        for arg in expr.children():
                            if is_int_value(arg):   # If it's a constant
                                constant_part += arg.as_long()
                            else:                   # If it's a symbolic variable
                                symbolic_part = arg
                    else:
                        # If expr is just the variable itself with no constant part
                        symbolic_part = expr
                
                    print(symbolic_part)
                    print(constant_part)
                
                
                if value is None:
                    print(f"Variable '{expression}' not initialized in current state.")
                    value = z3.Int(expression + "_0")  # e.g., x_0
                    current_state.add_variable(expression, "int", value)
                    
                if operator == '++':
                    if is_expr(value):
                        value = constant_part +1
                        exp = symbolic_part + value
                        new_value = simplify(exp)
                    else:
                        new_value = value + 1
                        
                    current_state.update_variable(expression, new_value)  # Update value in state
                    return new_value
                
                if operator == '--':
                    if is_expr(value):
                        value = constant_part -1
                        exp = symbolic_part + value
                        new_value = simplify(exp)
                    else:
                        new_value = value + 1
                        
                    current_state.update_variable(expression, new_value)  # Update value in state
                    return new_value
                
                if operator == '-':
                    if is_expr(value):
                        exp = -value
                        new_value = simplify(exp)
                    else:
                        new_value = -1*value
                        
                    current_state.update_variable(expression, new_value)
                else:
                    raise Exception(f"Unknown unary operator: {operator}")



        # If node type isn't handled, raise an exception
        else:
            raise Exception(f"Unknown expression type: {node.type}")
    
    
    
def mainFunction():
    
    with open("C:\\Users\\nitas\\Downloads\\csci591-007-WIMFG\\SampleCCode.c", "r") as f:
        content = f.read()
        
    engine = SymbolicExecutionEngine(content, 'test')
    engine.interpret()
    engine.output_results()
   
mainFunction()

Founnd the Function
Function node type: function_definition
child node type: primitive_type
child node type: function_declarator
Found function definition. To find if it has parameters
Parameter: b'test(int x)'
Parameters found:
Parameter: x of type int
State 0: {'x': 'int'} {'x': x_0} []
child node type: compound_statement
Found compound statement. Now processing
Found compound statement of type: compound_statement
Processing statement of type: {
Unsupported statement type: {
Processing statement of type: declaration
State Stack is not empty
Assignment node: init_declarator | b'y = f()'
Variable Type: int
Variable Name: y
Expression node: call_expression | b'f()'
Node Type inside Interpret: call_expression
Node Details :b'f()'
Since its fucntion call, skipping the function call for now
State 0: {'x': 'int', 'y': 'int'} {'x': x_0, 'y': y_0} []
Processing statement of type: if_statement
Inside Handle If Function
Node Type inside Interpret: parenthesized_expression
Node Details :b'(x > y